In [1]:
# Fetching the data
from sklearn.datasets import fetch_20newsgroups
import numpy as np
newsgroups = fetch_20newsgroups(subset='test')

# getting the topic ids
topic_idx = np.array(newsgroups.target, dtype=int)

# getting the unique topic names
topic_names = np.array(newsgroups.target_names)

# getting the list of documents
documents = list(newsgroups.data)

# getting the list of topics (in the same order as documents)
topics = list(topic_names[topic_idx])

In [2]:
topic_names

array(['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc',
       'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
       'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles',
       'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt',
       'sci.electronics', 'sci.med', 'sci.space',
       'soc.religion.christian', 'talk.politics.guns',
       'talk.politics.mideast', 'talk.politics.misc',
       'talk.religion.misc'], dtype='<U24')

In [3]:
import operator

def count_word_frequence(document,k):
  doc = document
  ##create a dist
  data = {}
  words = doc.replace("(",' ').replace(")",' ').replace("'",' ').replace("*",' ').replace(".",' ').replace("/",' ').replace("-",' ').replace(":",' ').replace("<",' ').replace(">",' ').replace("[",' ').replace("]",' ').replace("!",' ').replace("?",' ').replace(",",' ').strip("\n").lower()
  ##split the text
  words1 = words.split()
  ##count text
  for word in words1:
    if word in data:
      data[word] +=1
    else:
      data[word] = 1
  ###
  #sorted the dist by value in acending order
  sorted_fre = np.array(sorted(data.items(), key=operator.itemgetter(1),reverse=True))

  num_of_words = sorted_fre.shape[0]
  fre = np.zeros(num_of_words)
  words = []

  for i in range(0,num_of_words):
    fre[i] = int(sorted_fre[i][1])
    w = sorted_fre[i][0]
    words.append(w)
    
  words = np.array(words)
  

  return words[:k].tolist(),fre[:k].tolist()

In [4]:
topic_names_List = topic_names.tolist()
def getData(topic_names_List,k):
  data_dict_words_and_wordFre = {}
  for i in range(len(topic_names_List)):
    words,words_fre = count_word_frequence(document=documents[i],k=k)
    data_dict_words_and_wordFre[topic_names_List[i]] = {'words':words,'words_fre':words_fre}
    data_dict = {}

  for i in range(len(topic_names_List)):
    data_dict[topic_names_List[i]] = []
    for j in range(5):
      dic = {'word':data_dict_words_and_wordFre[topic_names_List[i]]['words'][j],'fre':data_dict_words_and_wordFre[topic_names_List[i]]['words_fre'][j]}
      data_dict[topic_names_List[i]].append(dic)
  return data_dict,data_dict_words_and_wordFre

In [5]:
data_dict,data_dict_words_and_wordFre=getData(topic_names_List,5)

In [6]:
from IPython.display import display, HTML
import json

In [7]:
%%javascript
require.config({
    paths: {
        d3: "https://d3js.org/d3.v6.min"
     }
});

require(["d3"], function(d3) {
    window.d3 = d3;
});

<IPython.core.display.Javascript object>

In [8]:
from string import Template

In [36]:
def getDoc():
    doc= {}
    for i in range(len(topic_names_List)):
        doc[topic_names_List[i]] = documents[i]
    return doc

In [10]:
name = ''
def target_func(comm, open_msg):
    # comm is the kernel Comm instance

    # Register handler for later messages
    @comm.on_msg
    def _recv(msg):
        # Use msg['content']['data'] for the data in the message
        document_name = msg['content']['data']['document_name']
        global name
        name = document_name
        #comm.send({'array':doc[n] })

get_ipython().kernel.comm_manager.register_target('my_comm_target', target_func)

In [41]:
print(name)

rec.sport.baseball


In [22]:
template = Template("""
<html>

<head>
</head>

<body>
  <!-- Load d3.js -->
  <script src="https://d3js.org/d3.v4.js"></script>

  <!-- Initialize a select button -->
  <select id="selectButton"></select>

  <!-- Create a div where the graph will take place -->
  <div id="my_dataviz"></div>

  <button id="button">Exported Document</button>

  <!-- Color Scale -->
  <script src="https://d3js.org/d3-scale-chromatic.v1.min.js"></script>
  <script>
    //dataset
    //console.log(name);
    var name = 'alt.atheism'
    // List of groups (here I have one group per column)
    var topic = $topic;
    //console.log(topic);

    var dict1 = $dict1;
    var dict2 = $dict2;
    var data1 = dict1[name];
    //console.log(dict1);

    var words = dict2[name]['words'];
    //console.log(words);
    
    var words_fre = dict2[name]['words_fre'];
    var max = words_fre[0]

    // add the options to the button
    d3.select("#selectButton")
      .selectAll('myOptions')
      .data(topic)
      .enter()
      .append('option')
      .text(function (d) { return d; }) // text showed in the menu
      .attr("value", function (d) { return d; }) // corresponding value returned by the button
    
    // set the dimensions and margins of the graph
    var margin = {top: 20, right: 30, bottom: 40, left: 90},
        width = 460 - margin.left - margin.right,
        height = 400 - margin.top - margin.bottom;

    // append the svg object to the body of the page
    var svg = d3.select("#my_dataviz")
                .append("svg")
                .attr("id","graph")
                .attr("width", width + margin.left + margin.right)
                .attr("height", height + margin.top + margin.bottom)
                .append("g")
                .attr("transform",
                "translate(" + margin.left + "," + margin.top + ")");

    // Initialize the X axis
    var x = d3.scaleLinear()
      .range([ 0, width]);
    var xAxis = svg.append("g")
      .attr("transform", "translate(0," + height + ")")
      .attr("class", "myXaxis");

    // Y axis
    var y = d3.scaleBand()
      .range([ 0, height ])
      .padding(.1);
    var yAxis = svg.append("g")

    function update(data_dict,data_word_and_fre){
      // Update the Y axis
      y.domain(data_word_and_fre['words']);
      yAxis.transition().duration(1000).call(d3.axisLeft(y));

      // Update the X axis
      max = data_word_and_fre['words_fre'][0];
      x.domain([0,max])
      xAxis.call(d3.axisBottom(x));

      //Bars
      var bar = svg.selectAll("myRect")
        .data(data_dict)
      
      bar
        .enter()
        .append("rect")
        .transition()
        .duration(1000)
          .attr("x", x(0) )
          .attr("id","myrect")
          .attr("y", function(d) { return y(d.word); })
          .attr("width", function(d) { return x(d.fre); })
          .attr("height", y.bandwidth() )
          .attr("fill", "#69b3a2");

      bar.exit()
        .remove()
    
      output()
    }
    
    // When the button is changed, run the updateChart function
    d3.select("#selectButton").on("change", function(d) {
        // recover the option that has been chosen
        var selectedOption = d3.select(this).property("value")
        // run the updateChart function with this selected option
        d3.select("#my_dataviz").selectAll("#myrect").remove();
        
        ////////////////////////////////
         
        
        
        ////////////////////////////////

        update(dict1[selectedOption],dict2[selectedOption]);
    })

    var first=true;
    if(first==true){
      update(dict1['alt.atheism'],dict2['alt.atheism']);
      fist=false;
    }
    
    function output(){
        let comm = Jupyter.notebook.kernel.comm_manager.new_comm('my_comm_target')
        // Send data
        comm.send({'document_name': 'alt.atheism'});

        // Register a handler
        comm.on_msg(function(msg) {
            //let data = msg.content.data.array;        
            //console.log(data)
            //d3.select("#div_receive_data").selectAll("*").remove()
            //bar_chart("#div_receive_data", data)
            //document.getElementById("div_receive_data").innerHTML = data;

        });

        // Setting up button
        document.getElementById("button").addEventListener("click", ()=>{
            let n = d3.select("#selectButton").property("value");

            console.log(n);
            comm.send({'document_name': n});

        }); 
    }

  </script>

</body>
</html>
""")
my_html = template.safe_substitute(topic=topic_names_List,dict1=data_dict,dict2=data_dict_words_and_wordFre)
display(HTML(my_html))

In [39]:
def get_exported_documents():
    global name
    doc = getDoc()
    
    return doc[name]

In [40]:
print(get_exported_documents())

From: tmc@spartan.ac.BrockU.CA (Tim Ciceran)
Subject: Re: Turning photographic images into thermal print and/or negatives
Organization: Brock University, St. Catharines Ontario
X-Newsreader: TIN [version 1.1 PL9]
Lines: 22

Jennifer Lynn Urso (ju23+@andrew.cmu.edu) wrote:
:  
: well, i have lots of experience with scanning in images and altering
: them.  as for changing them back into negatives, is that really possible?

: (stuff deleted)

: jennifer urso:  the oh-so bitter woman of utter blahness(but cheerful
: undertones)

I use Aldus Photostyler on the PC and I can turn a colour or black and white
image into a negative or turn a negative into a colour or black and white
image.  I don't know how it does it but it works well.  To test it I scanned
a negative and used Aldus to create a positive.  It looked better than the
print that the film developers gave me.


-- 

TMC
(tmc@spartan.ac.BrockU.ca)


